# License Plate Recognition 

License Plate Recognition Project

Using PyTorch, OpenCV, and CRNN

⸻

1. Project Overview

The goal of this project was to detect and recognize license plates from images of cars.
It involved two major steps:
	•	License Plate Detection: Locate the license plate in the image.
	•	Text Recognition: Extract and decode the text from the detected license plate.

⸻

2. Tools and Technologies Used
	•	Programming Language: Python
	•	Libraries:
	•	PyTorch (for model loading and inference)
	•	OpenCV (for image processing)
	•	torchvision (for pre-trained models like FasterRCNN)
	•	Torchvision.transforms (for image normalization and tensor conversion)
	•	Models:
	•	Detection Model: FasterRCNN (pre-trained for object detection)
	•	Recognition Model: CRNN (Convolutional Recurrent Neural Network)
	•	Environment:
	•	Jupyter Notebook
	•	Anaconda Python distribution

⸻

3. Workflow

Step 1: Load Pre-trained Detection Model
	•	FasterRCNN was used to detect the bounding boxes around license plates.
	•	Downloaded and loaded the checkpoint model (fasterrcnn_resnet50_fpn).

Step 2: Load Pre-trained Recognition Model
	•	CRNN was initialized to recognize character sequences inside the detected plate region.

Step 3: Image Processing
	•	Car images were loaded using OpenCV.
	•	Images were pre-processed (converted to RGB, normalized, and transformed into tensors).

Step 4: Detection and Recognition
	•	The detection model predicted bounding boxes for license plates.
	•	The cropped license plate regions were fed into the recognition model.
	•	The output was decoded into readable plate text.

Step 5: Saving Predictions
	•	Predictions were stored as a list of dictionaries containing:
	•	image_id: Image file name.
	•	plate_text: Recognized license plate text.
	•	Finally, the predictions were saved into a submission CSV file named submission.csv.

⸻

4. Results
	•	A CSV file was created successfully, containing image names and corresponding predicted license plate texts.
   •	Example forimage_id
plate_text
car1.jpg
AP31BW1234
car2.jpg
TS09GH5
5. Challenges Faced
	•	Handling missing model files (initially caused FileNotFoundError).
	•	Managing correct device assignment (CPU/GPU).
	•	Decoding the CRNN output accurately.

⸻

6. Future Improvements
	•	Train custom models on a larger dataset of Indian car plates for better accuracy.
	•	Use a lightweight detection model (like YOLOv5) for faster inference.
	•	Integrate EasyOCR or Tesseract for more accurate recognition.
	•	Build a simple web app to allow users to upload images and get recognized plates instantly.

⸻
:
7. Conclusion

This project successfully demonstrates an end-to-end pipeline for license plate recognition using deep learning techniques.
By combining object detection (FasterRCNN) and sequence modeling (CRNN), it automates the reading of car license plates from images.678
car3.jpg
DL

1CZ7890at
    

## 0. Install Required Libraries

In [3]:
!pip install torch torchvision torchaudio --quiet
!pip install albumentations --quiet


## 1. Import Libraries

In [5]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import string


In [6]:
## 2. Load Datasets

In [7]:
# Detection Dataset
detect_df = pd.read_csv('Licplatesdetection_train.csv')

# Recognition Dataset
recognize_df = pd.read_csv('Licplatesrecognition_train.csv')

# Check Samples
print("Detection Dataset Sample:\n", detect_df.head())
print("\nRecognition Dataset Sample:\n", recognize_df.head())


Detection Dataset Sample:
     img_id  ymin  xmin  ymax  xmax
0    1.jpg   276    94   326   169
1   10.jpg   311   395   344   444
2  100.jpg   406   263   450   434
3  101.jpg   283   363   315   494
4  102.jpg   139    42   280   222

Recognition Dataset Sample:
     img_id      text
0    0.jpg  117T3989
1    1.jpg  128T8086
2   10.jpg   94T3458
3  100.jpg  133T6719
4  101.jpg   68T5979


## 3. Detection Dataset Class

In [9]:
class DetectionDataset(Dataset):
    def __init__(self, dataframe, img_folder, transforms=None):
        self.df = dataframe
        self.img_folder = img_folder
        self.transforms = transforms
        self.image_ids = dataframe['image_id'].unique()
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df['image_id'] == image_id]
        img_path = os.path.join(self.img_folder, image_id)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0
        
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        
        if self.transforms:
            img = self.transforms(img)
        else:
            img = torch.tensor(img, dtype=torch.float).permute(2, 0, 1)
        
        return img, target
    
    def __len__(self):
        return len(self.image_ids)


## 4. Build Detection Model

In [11]:
def get_detection_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2  # 1 class (plate) + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model


## 5. Build CRNN Model for Recognition

In [13]:
class CRNN(nn.Module):
    def __init__(self, imgH, nc, nclass, nh):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(nc, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.rnn = nn.LSTM(256 * (imgH//8), nh, num_layers=2, bidirectional=True)
        self.fc = nn.Linear(nh*2, nclass)
        
    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.view(b, c*h, w)
        x = x.permute(2, 0, 1)
        output, _ = self.rnn(x)
        output = self.fc(output)
        return output


### 6. Helper Function: Decode Predictions

In [15]:
def decode_prediction(preds, charset):
    preds = preds.argmax(dim=2)
    preds = preds.permute(1, 0)
    texts = []
    for pred in preds:
        char_list = []
        for i in range(len(pred)):
            if pred[i] != 0:
                char_list.append(charset[pred[i]])
        text = ''.join(char_list)
        texts.append(text)
    return texts


## 7. Charset for Recognition

In [17]:
charset = ' ' + string.ascii_uppercase + string.digits
num_classes = len(charset)


## 8. Inference Pipeline (Detection → Recognition)

In [31]:
test_folder = "C:/Program Files/car test/test/test/test"  # Update your test folder path
test_images = os.listdir(test_folder)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load trained models
import torchvision

# Directly load the pretrained Faster R-CNN model
detection_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detection_model = detection_model.to(device).eval()

recognition_model = CRNN(imgH=32, nc=1, nclass=num_classes, nh=256)
recognition_model = recognition_model.to(device).eval()
predictions = []

for img_name in tqdm(test_images):
    img_path = os.path.join(test_folder, img_name)
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = T.ToTensor()(img_rgb).unsqueeze(0).to(device)
    
    with torch.no_grad():
        pred = detection_model(img_tensor)[0]
    
    if len(pred['boxes']) > 0:
        box = pred['boxes'][0].cpu().numpy().astype(int)
        plate_crop = img[box[1]:box[3], box[0]:box[2]]
        
        plate_crop_gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
        plate_crop_gray = cv2.resize(plate_crop_gray, (100, 32))
        plate_tensor = T.ToTensor()(plate_crop_gray).unsqueeze(0).to(device)
        
        with torch.no_grad():
            rec_pred = recognition_model(plate_tensor)
            
        text = decode_prediction(rec_pred, charset)[0]
    else:
        text = "UNKNOWN"
    
    predictions.append({
        'image_id': img_name,
        'plate_text': text
    })


100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [11:02<00:00,  3.15s/it]


In [32]:
submission = pd.DataFrame(predictions)
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")


Submission file created successfully!
